In [20]:
import numpy as np
import pandas as pd
from pandas_datareader import data as wb
import matplotlib.pyplot as plt
%matplotlib inline

from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

from keras.models import load_model

In [27]:
EPOCHS = 300
h1 = 200
h2 = 100

stock_list = ["AAPL","MSFT"]

num_stocks = len(stock_list)
lookback = 30
predictions = 10
periods = 20

In [3]:
def get_data(ticker):
    df = wb.DataReader(ticker, "yahoo", "2010-01-01")
    df = df[["Adj Close"]]
    return df

In [4]:
def plot_data(*dfs):
    for df in dfs:
        plt.plot(df, label = f"stock_list{i for i in stock_list}")
    plt.legend(loc = 4)
    plt.show()

In [5]:
stocks = [ get_data(ticker) for ticker in stock_list]

In [9]:
scaler = MinMaxScaler()
args = [ df.values.reshape(df.shape[0],1) for df in stocks]

n_args = len(args)

In [10]:
np_arr = scaler.fit_transform(np.concatenate((args[0],args[1]), axis = 1))

In [15]:
div = len(np_arr) - periods * predictions

train = np_arr[: div]
test = np_arr[div - lookback:]

In [16]:
test

array([[0.64971501, 0.70454294],
       [0.64131972, 0.70978956],
       [0.61477023, 0.67556933],
       [0.61950053, 0.69726035],
       [0.61536682, 0.70509108],
       [0.61736975, 0.70892814],
       [0.6252536 , 0.71182552],
       [0.60194286, 0.68504442],
       [0.59538012, 0.66045602],
       [0.60445723, 0.66891319],
       [0.58238238, 0.66672062],
       [0.56508049, 0.64964972],
       [0.53908488, 0.62396491],
       [0.52246487, 0.591859  ],
       [0.56652934, 0.64221052],
       [0.56218261, 0.6470655 ],
       [0.56252348, 0.64087926],
       [0.56895849, 0.6501195 ],
       [0.56972558, 0.64659572],
       [0.50269125, 0.61746544],
       [0.52855887, 0.65293857],
       [0.5271526 , 0.65395655],
       [0.5391701 , 0.65975126],
       [0.55007969, 0.67126244],
       [0.55216791, 0.66601581],
       [0.54573297, 0.65975126],
       [0.53597398, 0.65387831],
       [0.54905701, 0.6770572 ],
       [0.55702614, 0.67995447],
       [0.56094678, 0.68574924],
       [0.

In [17]:
train

array([[0.01004369, 0.04539697],
       [0.01180722, 0.04833676],
       [0.01200618, 0.04839927],
       ...,
       [0.54905701, 0.6770572 ],
       [0.55702614, 0.67995447],
       [0.56094678, 0.68574924]])

In [19]:
def preprocess( df, lookback, prediction_days, n_stocks, jump =1):
    x,y = [],[]
    for i in range(0, len(df)-lookback-prediction_days + 1, jump):
        x.append(df[i:(i+lookback)])
        y.append(df[(i+lookback):(i + lookback + prediction_days)])
    return np.array(x), np.array(y)

In [21]:
x_test, y_test = preprocess(test, lookback, predictions, num_stocks, predictions)
y_test = np.array([list(a.ravel()) for a in y_test])

x, y = preprocess(train, lookback, predictions, num_stocks)
y = np.array([list(b.ravel()) for b in y])

x_train , x_val , y_train, y_val = train_test_split(x, y, test_size = 0.30, random_state = 51)

In [26]:
#Dimensions - (Sample,Timestep,Features)
print(x_train.shape)
print(x_val.shape)
print(x_test.shape)
print(y_train.shape)
print(y_val.shape)
print(y_test.shape)

(1566, 30, 2)
(672, 30, 2)
(20, 30, 2)
(1566, 20)
(672, 20)
(20, 20)


In [28]:
model = Sequential()

model.add(LSTM(h1, input_shape = (lookback,num_stocks),return_sequences = True))
model.add(LSTM(h2, input_shape = (h1, 1)))
model.add(Dense(predictions * num_stocks))
model.compile(loss = "mean_squared_error", optimizer = "adam")

Instructions for updating:
Colocations handled automatically by placer.


In [ ]:
Sess = model.fit(x_train,y_train,epochs=EPOCHS,validation_data=(x_val,y_val),shuffle=True,batch_size=1, verbose=2)

Instructions for updating:
Use tf.cast instead.
Train on 1566 samples, validate on 672 samples
Epoch 1/300
 - 199s - loss: 0.0017 - val_loss: 5.7387e-04
Epoch 2/300
 - 190s - loss: 8.8688e-04 - val_loss: 5.1451e-04
Epoch 3/300
 - 200s - loss: 5.9309e-04 - val_loss: 3.4571e-04
Epoch 4/300
 - 196s - loss: 5.3109e-04 - val_loss: 3.6451e-04
Epoch 5/300
 - 191s - loss: 4.8260e-04 - val_loss: 2.7995e-04
Epoch 6/300
 - 191s - loss: 4.7793e-04 - val_loss: 3.2214e-04
Epoch 7/300
 - 197s - loss: 4.0267e-04 - val_loss: 6.7442e-04
Epoch 8/300
 - 191s - loss: 3.7447e-04 - val_loss: 7.2169e-04
Epoch 9/300
 - 193s - loss: 3.5978e-04 - val_loss: 2.7101e-04
Epoch 10/300
 - 191s - loss: 3.4634e-04 - val_loss: 2.7034e-04
Epoch 11/300
 - 192s - loss: 3.5342e-04 - val_loss: 3.5068e-04
Epoch 12/300
 - 192s - loss: 3.4354e-04 - val_loss: 3.3950e-04
Epoch 13/300
 - 194s - loss: 3.2978e-04 - val_loss: 2.5861e-04
Epoch 14/300
 - 189s - loss: 3.1435e-04 - val_loss: 4.5012e-04
Epoch 15/300
 - 193s - loss: 3.3075e

In [ ]:
plt.figure(figsize = (13,10))
plt.plot(sess.sess["loss"],label = "Loss")
plt.plot(sess.sess["val_loss"], label = "Val Loss")
plt.legend(loc = 4)
plt.show()

In [ ]:
x_test , y_test = preprocess(test , lookback, predictions, num_stocks, predictions)
y_test = np.array([list(a.ravel() for a in y_test)])

x_h = model.predict(x_test)
x_h.shape()

In [ ]:
def inverse_transformation(results,num_stocks):
    #From input/output nootbook: apply makeup, use scl.inverse_transform and remove makeup
    
    #transform to input shape
    original_matrix_format = []
    for result in results:
        #do inverse transform
        original_matrix_format.append(scl.inverse_transform([result[x:x+num_companies] for x in range(0, len(result), num_stocks)]))
    original_matrix_format = np.array(original_matrix_format)
    
    #restore to original shape
    for i in range(len(original_matrix_format)):
        results[i] = original_matrix_format[i].ravel()

    return output_result

In [ ]:
def prediction_by_company(raw_model_output, num_stocks):
    matrix_prediction = []
    for i in range(0,num_stocks):
        matrix_prediction.append([[lista[j] for j in range(i,len(lista),num_stocks)] for lista in raw_model_output])
    return np.array(matrix_prediction)

In [ ]:
x_h = inverse_transformation(x_h,num_stocks)

x_h1 = prediction_by_company(x_h, num_stocks)
x_h1.shape

In [ ]:
def target_by_company(raw_model_output, num_stocks):
    matrix_target = [[] for x in range(num_stocks)]
    
    for output in raw_model_output:
        for i in range (num_stocks):
            for j in range(0,len(output),num_stocks):
                matrix_target[i].append(output[i+j])
    return np.array(matrix_target)

In [ ]:
y_h = inverse_transformation(y_test)

y_h1 = target_by_company(y_h, num_stocks)
y_h1.shape